In [1]:
import numpy as np
import numba
import matplotlib.pyplot as plt
import sympy as sym
import scipy.optimize as sopt
#plt.style.use('presentation')
%matplotlib notebook

def d2np(d):
    
    names = []
    numbers = ()
    dtypes = []
    for item in d:
        names += item   
        if type(d[item]) == float:
            numbers += (d[item],)
            dtypes += [(item,float)]
        if type(d[item]) == int:
            numbers += (d[item],)
            dtypes += [(item,int)]
        if type(d[item]) == np.ndarray:
            numbers += (d[item],)
            dtypes += [(item,np.float64,d[item].shape)]
    return np.array([numbers],dtype=dtypes)

In [2]:
alpha = np.exp(2.0/3*np.pi*1j)
A_0a =  np.array([[1, 1, 1],
                  [1, alpha**2, alpha],
                  [1, alpha, alpha**2]])

A_a0 = 1/3* np.array([[1, 1, 1],
                      [1, alpha, alpha**2],
                      [1, alpha**2, alpha]])

In [3]:
V_0 = 0.0*np.exp(1j*0)
V_1 = 400/np.sqrt(3)*np.exp(1j*0)
V_2 = 0.0*np.exp(1j*0)
V_012 = np.array([V_0,V_1,V_2]).reshape(3,1)

In [4]:
V_abc = A_0a @ V_012

In [5]:
V_abc

array([[ 230.94010768  +0.j],
       [-115.47005384-200.j],
       [-115.47005384+200.j]])

In [6]:
I_0 = 0.0*np.exp(1j*0)
I_1 = 10*np.exp(1j*0)
I_2 = 0.0*np.exp(1j*0)
I_012 = np.array([I_0,I_1,I_2]).reshape(3,1)

In [7]:
I_abc = A_0a @ I_012

In [8]:
Z_abc = np.eye(3)*30
Y_abc = np.linalg.inv(Z_abc)

In [9]:
#V_abc = Z_abc @ I_abc

In [10]:
I_abc = Y_abc @ V_abc

In [11]:
I_abc

array([[ 7.69800359+0.j        ],
       [-3.84900179-6.66666667j],
       [-3.84900179+6.66666667j]])

In [12]:
Y_abc

array([[ 0.03333333,  0.        ,  0.        ],
       [ 0.        ,  0.03333333,  0.        ],
       [ 0.        ,  0.        ,  0.03333333]])

In [13]:
Y_0 = 0.3
Y_1 = 0.0
Y_2 = 0.0

Y_012 = np.array([[ Y_0,  0. ,  0. ],
                   [ 0. , Y_1,  0. ],
                   [ 0. ,  0. ,  Y_2]])


In [14]:
Y_abc = A_0a @ Y_012 @ A_a0

In [15]:
I_abc = Y_abc @ V_abc

In [16]:
I_abc

array([[ -1.77635684e-15 +7.10542736e-15j],
       [ -1.77635684e-15 +7.10542736e-15j],
       [ -1.77635684e-15 +7.10542736e-15j]])

In [17]:
V_0 = 0.0*np.exp(1j*0)
V_1 = 400/np.sqrt(3)*np.exp(1j*0)
V_2 = 0.0*np.exp(1j*0)
V_012 = np.array([V_0,V_1,V_2]).reshape(3,1)

I_abc = Y_abc @ V_abc



In [18]:
Y_abc

array([[ 0.1+0.j,  0.1+0.j,  0.1+0.j],
       [ 0.1+0.j,  0.1+0.j,  0.1+0.j],
       [ 0.1+0.j,  0.1+0.j,  0.1+0.j]])

In [54]:
Z_gnd = 32.0
Y_gnd = 1.0/Z_gnd 
Y_abcn = np.array([[ 1,  1,  1, -Y_gnd],
                   [ 1,  1,  1, -Y_gnd],
                   [ 1,  1,  1, -Y_gnd],
                   [ 1,  1,  1,  3*Y_gnd]])

V_0 = 0.0*np.exp(1j*0)
V_1 = 20e3/np.sqrt(3)*np.exp(1j*0)
V_2 = 0.0*np.exp(1j*0)
V_012 = np.array([V_0,V_1,V_2]).reshape(3,1)
V_abc = A_0a @ V_012

V_abcn = np.vstack((V_abc,[11e3+1j*0.0]))

I_abcn = Y_abcn @ V_abcn
I_abcn

array([[ -343.75 +3.63797881e-12j],
       [ -343.75 +3.63797881e-12j],
       [ -343.75 +3.63797881e-12j],
       [ 1031.25 +3.63797881e-12j]])

In [50]:
V_abcn

array([[ 230.94010768  +0.j],
       [-115.47005384-200.j],
       [-115.47005384+200.j],
       [   0.00000000  +0.j]])

In [ ]:
## Kron reduction

In [199]:
Z_gnd = 32.0*3
Y_gnd = 1.0/Z_gnd 
Y_full = -Y_gnd/3+Y_gnd
Y_gnd1 = Y_gnd
Y_gnd2 = Y_gnd*3
Y_abcn = np.array([[  Y_full+1e-6,  Y_full,  Y_full, -Y_gnd1],
                   [  Y_full,  Y_full+1e-6,  Y_full, -Y_gnd1],
                   [  Y_full,  Y_full,  Y_full+1e-6, -Y_gnd1],
                   [ -Y_gnd1, -Y_gnd1, -Y_gnd1,  Y_gnd2]])
Y_pp = Y_abcn[0:3,0:3]
Y_pn = Y_abcn[0:3,3:4]
Y_np = Y_abcn[3:4,0:3]
Y_nn = Y_abcn[3:4,3:4]
Y_abc = Y_pp - Y_pn @ np.linalg.inv(Y_nn) @ Y_np
Z_abc = np.linalg.inv(Y_abc)
V_0 = 0e3/np.sqrt(3)*np.exp(1j*0)
V_1 = 0e3/np.sqrt(3)*np.exp(1j*0)
V_2 = 20.0*np.exp(1j*0)
V_012 = np.array([V_0,V_1,V_2]).reshape(3,1)
V_abc = A_0a @ V_012

I_abc = Y_abc @ V_abc
I_abc

array([[  2.00000000e-05 +2.77555756e-17j],
       [ -1.00000000e-05 +1.73205081e-05j],
       [ -1.00000000e-05 -1.73205081e-05j]])

In [200]:
20e3/np.sqrt(3)/32

360.8439182435161

In [201]:
360.0*2

720.0

In [202]:
360*3

1080

In [203]:
np.linalg.inv(Y_abc)

array([[ 666698.66359481, -333301.33640496, -333301.33640496],
       [-333301.33640496,  666698.66359486, -333301.33640501],
       [-333301.33640496, -333301.33640501,  666698.66359486]])